In [ ]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/gaze_estimation'


In [ ]:
#@title Import required modules

import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression


from hglm.precision_module import large_precision_module
from loss.h_likelihood_precision import nhll_correlated_precision
from util import make_reproducibility, TensorDataset, convert_to_xyz, mae
from networks import *
from hglm.hglm_correlated_precision import correlated_precision_without_val

# Data Load

In [ ]:
device = torch.device('cuda:0')
seed = 10

experiment_name = 'ut_3fold'
batch_size = 1000
pretrain_iter = 1
m_pretrain_epoch = 50
v_pretrain_epoch = 20
max_iter = 150
mean_epoch = 10
v_step_iter = 100
patience = 10
mean_lr = 5e-3
variance_lr = 1e-3
weight_decay = 0
hidden_features = 500
test_unseen=True
large_test=True

In [ ]:
res_list = []

In [ ]:
#@title test

for fold in range(3) : 

    train_ids = np.load(f'../ut_dataset/3-fold/fold_{fold}_train_ids.npy').reshape(-1)
    train_images = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_images.npy'), dtype=torch.float)
    train_hps = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_2d_hps.npy'), dtype=torch.float)
    train_gazes = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_train_2d_gazes.npy'), dtype=torch.float)

    test_ids = np.load(f'../ut_dataset/3-fold/fold_{fold}_test_ids.npy').reshape(-1)
    test_images = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_images.npy'), dtype=torch.float)
    test_hps = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_2d_hps.npy'), dtype=torch.float)
    test_gazes = torch.as_tensor(np.load(f'../ut_dataset/3-fold/fold_{fold}_test_2d_gazes.npy'), dtype=torch.float)

    res_list.append(correlated_precision_without_val(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        ResNet_batchnorm.ResNet_batchnorm, hidden_features=hidden_features, K=2,
        mean_lr=mean_lr, variance_lr=variance_lr, weight_decay=weight_decay, batch_size=batch_size,
        pretrain_iter=pretrain_iter, m_pretrain_epoch=m_pretrain_epoch, v_pretrain_epoch=v_pretrain_epoch, max_iter=max_iter, mean_epoch=mean_epoch, v_step_iter=v_step_iter, patience=patience,
        device=device, experiment_name=f'{experiment_name}_{fold}', SEED=seed + fold,
        normalize=True, deg=True, test_unseen=test_unseen, large_test=large_test, weighted=True, verbose=False))


